In [0]:
import tensorflow as tf

In [0]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone dev https://github.com/fenwickslab/fenwicks.git

Cloning into 'fenwicks'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 495 (delta 44), reused 56 (delta 23), pack-reused 413
Receiving objects: 100% (495/495), 229.51 KiB | 4.17 MiB/s, done.
Resolving deltas: 100% (260/260), done.


In [0]:
from IPython.display import Audio
from scipy.io import wavfile
import librosa
import fenwicks as fw
import os
import numpy as np

In [0]:
BUCKET = 'gs://gs_colab' #@param {type:"string"}
PROJECT = 'speech_cmd' #@param {type:"string"}
BATCH_SIZE = 128 #@param {type: "integer"}
EPOCHS = 24 #@param {type:"integer"}
LEARNING_RATE = 0.001 #@param {type:"number"}

In [0]:
fw.colab_tpu.setup_gcs()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
data_dir, work_dir = fw.io.get_gcs_dirs(BUCKET, PROJECT)

In [0]:
data_dir_local = fw.datasets.untar_data(fw.datasets.URLs.SPEECH_CMD_001, './speech001')

In [0]:
Audio(os.path.join(data_dir_local, 'happy/012c8314_nohash_0.wav'))

In [0]:
SAMPLE_RATE = 16000
NOISE_DIR = '_background_noise_'
data_fn = os.path.join(data_dir, 'speech001_images')

In [0]:
def gen_silence():
  NUM_SEGMENT = 400
  NUM_SYNTH = 500

  path = os.path.join(data_dir_local, NOISE_DIR)
  out_path = os.path.join(data_dir_local, 'silence/')

  fw.io.create_clean_dir(out_path)  
  files = fw.io.enum_files(path, 'wav')
  
  for filename in files:
    _, samples = wavfile.read(filename)
    for i in range(NUM_SEGMENT):
      out_name = f'segment_{i}_{os.path.basename(filename)}'
      data = (samples[i * 200: i * 200 + SAMPLE_RATE] 
              * max(0, 2 * (np.random.random() - 0.25))).astype('int16')
      if data.max() != 0:
        wavfile.write(out_path + out_name, SAMPLE_RATE, data)

  for i in range(NUM_SYNTH):
    d = np.zeros(SAMPLE_RATE)
    loc = np.random.randint(0, SAMPLE_RATE, 4600)
    d[loc[::2]] = -1
    d[loc[1::2]] = 1
    wavfile.write(os.path.join(out_path, f'new_synthesized_{i}.wav'), SAMPLE_RATE, d)

In [0]:
gen_silence()

/usr/local/lib/python3.6/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [0]:
def read_logmelspectrogram(filepath, n_components=40, n_fft=400, hop_length=160):
  samples, _ = librosa.load(filepath, sr=SAMPLE_RATE)
  samples = samples[:SAMPLE_RATE]
  samples = np.pad(samples, (0, max(0, SAMPLE_RATE - len(samples))), 'constant')
  melspectrogram = librosa.feature.melspectrogram(y=samples, sr=SAMPLE_RATE, 
    n_fft=n_fft, hop_length=hop_length, n_mels=n_components)
  return librosa.power_to_db(melspectrogram)

In [0]:
paths, y, labels = fw.io.data_dir_tfrecord(data_dir_local, data_fn,
                  extractor=read_logmelspectrogram, 
                  file_ext='wav', exclude_dirs=[NOISE_DIR])

In [0]:
n_classes = len(labels)
n_all = len(y)

In [0]:
def build_vgg_blk(c, kernel_size):
  blk = fw.Sequential()
  blk.add(fw.layers.ConvBN(c, kernel_size))
  blk.add(fw.layers.ConvBN(c, kernel_size))
  blk.add(tf.keras.layers.MaxPooling2D())
  return blk

In [0]:
def build_nn(c=16, kernel_size=(2,5), c_dense=256, drop_rate=0.5):
  model = fw.Sequential()
  model.add(build_vgg_blk(c, kernel_size))
  model.add(build_vgg_blk(c*2, kernel_size))
  model.add(build_vgg_blk(c*4, kernel_size))
  model.add(build_vgg_blk(c*8, kernel_size))
  model.add(fw.layers.GlobalPools())
  model.add(fw.layers.DenseBlk(c_dense, drop_rate))
  model.add(fw.layers.DenseBlk(c_dense, drop_rate))
  model.add(tf.keras.layers.Dense(n_classes, use_bias=False))
  return model

In [0]:
n_valid = n_all // 5 // 8 * 8
n_train = n_all - n_valid

total_steps = n_train // BATCH_SIZE * EPOCHS
opt_func = fw.train.adam_sgdr_one_cycle(total_steps, lr=LEARNING_RATE)
model_func = fw.tpuest.get_clf_model_func(build_nn, opt_func)

In [0]:
parser = lambda x: fw.io.tfexample_numpy_image_parser(x, 101, 40, 1)

est = fw.tpuest.get_tpu_estimator(n_train, n_valid, model_func, work_dir, trn_bs=BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/speech_cmd/2019-04-10-22:05:59', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.84.91.18:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f837cb734a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.84.91.18:8470', '_evaluation_master': 'grpc://10.84.91.18:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=418, num_shards=None,

In [0]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser,
  params['batch_size'], n_folds=5, val_fold_idx = 0, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser,
  params['batch_size'], n_folds=5, val_fold_idx = 0, training=False)

est.train(train_input_func, steps=total_steps)
result0 = est.evaluate(valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Querying Tensorflow master (grpc://10.84.91.18:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 12094140865951980020)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 330122869396192216)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3350169301198750023)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8156290480069505914)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 12266760561040780093)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/j

In [0]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser,
  params['batch_size'], n_folds=5, val_fold_idx = 1, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser,
  params['batch_size'], n_folds=5, val_fold_idx = 1, training=False)

est.train(train_input_func, steps=total_steps)
result1 = est.evaluate(valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://gs_colab/work/speech_cmd/2019-04-10-22:05:59/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 10 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (418) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (418) batch(es) of data from outfeed.
INFO:tensorflow:loss 

In [0]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser,
  params['batch_size'], n_folds=5, val_fold_idx = 2, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser,
  params['batch_size'], n_folds=5, val_fold_idx = 2, training=False)

est.train(train_input_func, steps=total_steps)
result2 = est.evaluate(valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://gs_colab/work/speech_cmd/2019-04-10-22:05:59/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 10 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (418) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (418) batch(es) of data from outfeed.
INFO:tensorflow:loss 

In [0]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser,
  params['batch_size'], n_folds=5, val_fold_idx = 3, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser,
  params['batch_size'], n_folds=5, val_fold_idx = 3, training=False)

est.train(train_input_func, steps=total_steps)
result3 = est.evaluate(valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://gs_colab/work/speech_cmd/2019-04-10-22:05:59/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 10 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (418) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (418) batch(es) of data from outfeed.
INFO:tensorflow:loss 

In [0]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser,
  params['batch_size'], n_folds=5, val_fold_idx = 4, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser,
  params['batch_size'], n_folds=5, val_fold_idx = 4, training=False)

est.train(train_input_func, steps=total_steps)
result4 = est.evaluate(valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://gs_colab/work/speech_cmd/2019-04-10-22:05:59/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 10 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (418) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (418) batch(es) of data from outfeed.
INFO:tensorflow:loss 

In [0]:
result0, result1, result2, result3, result4

({'accuracy': 0.9492981, 'global_step': 10032, 'loss': 0.16923833},
 {'accuracy': 0.9478793, 'global_step': 10032, 'loss': 0.18635637},
 {'accuracy': 0.9467593, 'global_step': 10032, 'loss': 0.19319052},
 {'accuracy': 0.9453405, 'global_step': 10032, 'loss': 0.22283693},
 {'accuracy': 0.9457885, 'global_step': 10032, 'loss': 0.20261507})